In [1]:
# подгружаем необходимые библиотеки и фиксируем random state

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import re
import scipy
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
# %matplotlib inline

RS=42

ноутбук с решением кейса от Сбербанка на хакатоне Цифровой Прорыв с 05 по 07 июня 2020 года. Капитан - Белоушкин Александр, дата саентист - Бочаров Алексей, дизайнер - Вера.

In [0]:
#подключаем гугл диск на котором данные
from google.colab import drive
drive.mount ('/content/gdrive', force_remount = True)

Mounted at /content/gdrive


In [2]:
#загружаем данные в датафреймы
bad_df=pd.read_csv('bad.csv',sep=';')
bad_df.head(1_000_000)

,id,address
0,20000001,"115280, г. Москва, ул. Восточная, д. 9 этаж 1 ..."
1,20000002,"115280, г. Москва, ул. Ленинская Слобода, д. 2..."
2,20000003,"117105, г. Москва, шоссе Варшавское, д. 1 стр...."
3,20000004,"117321, г. Москва, ул. Профсоюзная, д. 128, ко..."
4,20000005,"121357, г. Москва, ул. Нежинская, д. 1 корп. 4..."
5,20000006,"142211, Московская обл, г. Серпухов, шоссе Мос..."
6,20000007,"111674, г. Москва, ул. Вольская 2-Я, 22, 2, п..."
7,20000008,"119049, г. Москва, ул. Мытная, д. 28 стр. 3 эт..."
8,20000009,"129110, Москва, Москва, Москва, пер Орлово-Дав..."
9,20000010,"""109044, г.Москва, ул. Крутицкий Вал, дом 8/22..."


In [3]:
#загружаем данные в датафреймы
good_df=pd.read_csv('good_half2.csv',sep=';',index_col='num')
good_df.head(1_000_000)

,id,original,type,country,town,field8,field9,fiel10,field11,field12,field13,field14,field15,field16
num,,,,,,,,,,,,,,
1,10000003,"115516, Москва, Бехтерева, 2, 1",house,РОССИЯ,МОСКВА,МОСКВА,NaN,NaN,NaN,Улица,БЕХТЕРЕВА,2,NaN,NaN
2,10000004,"117279, г. Москва, ул. Миклухо-Маклая, д.34, ...",house,РОССИЯ,МОСКВА,МОСКВА,NaN,NaN,NaN,Улица,МИКЛУХО-МАКЛАЯ,34,NaN,NaN
3,10000005,"127083, Москва, ул. Масловка В, 20, 1, 6",house,РОССИЯ,МОСКВА,МОСКВА,NaN,NaN,NaN,Улица,МАСЛОВКА В.,20,NaN,1
4,10000006,"Область Московская, Район Раменский",subregion,РОССИЯ,МОСКОВСКАЯ,NaN,NaN,Район,РАМЕНСКИЙ,NaN,NaN,NaN,NaN,NaN
5,10000007,"Область Московская, Район Сергиево-Посадский,...",city,РОССИЯ,МОСКОВСКАЯ,ХОТЬКОВО,NaN,Район,СЕРГИЕВО-ПОСАДСКИЙ,NaN,NaN,NaN,NaN,NaN
6,10000008,"г.Москва,-,ул.Ляпидевского, вл.1",street,РОССИЯ,МОСКВА,МОСКВА,NaN,NaN,NaN,Улица,ЛЯПИДЕВСКОГО,NaN,NaN,NaN
7,10000009,"108841, г. Москва, г. Троицк, м-н ""В"", 54, 74",house,РОССИЯ,МОСКВА,ТРОИЦК,NaN,NaN,NaN,Микрорайон,В,54,NaN,NaN
8,10000010,"109145, Москва, Привольная, 2, 1, ПОМЕЩЕНИЕ 3;...",house,РОССИЯ,МОСКВА,МОСКВА,NaN,NaN,NaN,Улица,МАРШАЛА РЫБАЛКО,2,NaN,6
9,10000012,"125167 Москва г, Ленинградский пр-кт, 36 СТР.1...",house,РОССИЯ,МОСКВА,МОСКВА,NaN,NaN,NaN,Проезд,ЛИНЕЙНЫЙ,6А,NaN,NaN


In [4]:
good_df.rename(columns={'type':'detection_level',	'country':'country',	'town':'region',	'field8':'gorod',\
                        'field9':'mesto',	'fiel10':'region_type',	'field11':'rayon',	\
                        'field12':'type_street',	'field13':'street',	'field14':'dom'	,'field15':'kv',	'field16':'ofice'},\
               inplace=True)
good_df.head()

,id,original,detection_level,country,region,gorod,mesto,region_type,rayon,type_street,street,dom,kv,ofice
num,,,,,,,,,,,,,,
1,10000003,"115516, Москва, Бехтерева, 2, 1",house,РОССИЯ,МОСКВА,МОСКВА,NaN,NaN,NaN,Улица,БЕХТЕРЕВА,2,NaN,NaN
2,10000004,"117279, г. Москва, ул. Миклухо-Маклая, д.34, ...",house,РОССИЯ,МОСКВА,МОСКВА,NaN,NaN,NaN,Улица,МИКЛУХО-МАКЛАЯ,34,NaN,NaN
3,10000005,"127083, Москва, ул. Масловка В, 20, 1, 6",house,РОССИЯ,МОСКВА,МОСКВА,NaN,NaN,NaN,Улица,МАСЛОВКА В.,20,NaN,1
4,10000006,"Область Московская, Район Раменский",subregion,РОССИЯ,МОСКОВСКАЯ,NaN,NaN,Район,РАМЕНСКИЙ,NaN,NaN,NaN,NaN,NaN
5,10000007,"Область Московская, Район Сергиево-Посадский,...",city,РОССИЯ,МОСКОВСКАЯ,ХОТЬКОВО,NaN,Район,СЕРГИЕВО-ПОСАДСКИЙ,NaN,NaN,NaN,NaN,NaN


In [5]:
good_df.describe()

,id
count,6.833010e+05
mean,1.034169e+07
std,1.972676e+05
min,1.000000e+07
25%,1.017085e+07
50%,1.034170e+07
75%,1.051253e+07
max,1.068336e+07


In [6]:
#заполняем пустые ячейки
good_df.fillna(value='0',inplace=True)
good_df.head(1_000_000)

,id,original,detection_level,country,region,gorod,mesto,region_type,rayon,type_street,street,dom,kv,ofice
num,,,,,,,,,,,,,,
1,10000003,"115516, Москва, Бехтерева, 2, 1",house,РОССИЯ,МОСКВА,МОСКВА,0,0,0,Улица,БЕХТЕРЕВА,2,0,0
2,10000004,"117279, г. Москва, ул. Миклухо-Маклая, д.34, ...",house,РОССИЯ,МОСКВА,МОСКВА,0,0,0,Улица,МИКЛУХО-МАКЛАЯ,34,0,0
3,10000005,"127083, Москва, ул. Масловка В, 20, 1, 6",house,РОССИЯ,МОСКВА,МОСКВА,0,0,0,Улица,МАСЛОВКА В.,20,0,1
4,10000006,"Область Московская, Район Раменский",subregion,РОССИЯ,МОСКОВСКАЯ,0,0,Район,РАМЕНСКИЙ,0,0,0,0,0
5,10000007,"Область Московская, Район Сергиево-Посадский,...",city,РОССИЯ,МОСКОВСКАЯ,ХОТЬКОВО,0,Район,СЕРГИЕВО-ПОСАДСКИЙ,0,0,0,0,0
6,10000008,"г.Москва,-,ул.Ляпидевского, вл.1",street,РОССИЯ,МОСКВА,МОСКВА,0,0,0,Улица,ЛЯПИДЕВСКОГО,0,0,0
7,10000009,"108841, г. Москва, г. Троицк, м-н ""В"", 54, 74",house,РОССИЯ,МОСКВА,ТРОИЦК,0,0,0,Микрорайон,В,54,0,0
8,10000010,"109145, Москва, Привольная, 2, 1, ПОМЕЩЕНИЕ 3;...",house,РОССИЯ,МОСКВА,МОСКВА,0,0,0,Улица,МАРШАЛА РЫБАЛКО,2,0,6
9,10000012,"125167 Москва г, Ленинградский пр-кт, 36 СТР.1...",house,РОССИЯ,МОСКВА,МОСКВА,0,0,0,Проезд,ЛИНЕЙНЫЙ,6А,0,0


In [7]:
TOKEN_RE = re.compile(r'[\w\d]+')  #regular expression to start with

In [8]:
def tokenize_text_simple_regex(txt, min_token_size=2):
    """ This func tokenize text with TOKEN_RE applied ealier """
    txt = txt.lower()
    all_tokens = TOKEN_RE.findall(txt)
    return [token for token in all_tokens if len(token) >= min_token_size]

def tokenize_corpus(texts, tokenizer=tokenize_text_simple_regex, **tokenizer_kwargs):
    """
    This func tokenize corpus of docs
    """
    return [tokenizer(text, **tokenizer_kwargs) for text in texts]

In [9]:
X=good_df.original
X.head()

num
1                      115516, Москва, Бехтерева, 2, 1
2     117279, г. Москва, ул. Миклухо-Маклая, д.34, ...
3             127083, Москва, ул. Масловка В, 20, 1, 6
4                  Область Московская, Район Раменский
5     Область Московская, Район Сергиево-Посадский,...
Name: original, dtype: object

In [10]:
y=good_df[['detection_level','country','region','gorod']]
y.head()


,detection_level,country,region,gorod
num,,,,
1,house,РОССИЯ,МОСКВА,МОСКВА
2,house,РОССИЯ,МОСКВА,МОСКВА
3,house,РОССИЯ,МОСКВА,МОСКВА
4,subregion,РОССИЯ,МОСКОВСКАЯ,0
5,city,РОССИЯ,МОСКОВСКАЯ,ХОТЬКОВО


In [11]:
#Количество меток
y.detection_level.nunique()

5

In [12]:
y.country.nunique()

2

In [13]:
y.gorod.nunique()

385

In [14]:
from sklearn.ensemble import RandomForestClassifier


In [0]:
X_train, y_train, X_test, y_test=train_test_split(X, y, test_size=0.2) 

In [15]:
sklearn_pipeline = Pipeline((('vect', TfidfVectorizer(tokenizer=tokenize_text_simple_regex,
                                                      max_df=0.8,
                                                      min_df=5)),
                             ('cls', RandomForestClassifier(verbose=True,random_state=RS))))

In [16]:
sklearn_pipeline.fit(X[:1000], y[:1000]);

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.5s finished


In [17]:
sklearn_train_pred = sklearn_pipeline.predict_proba(X[:1000])

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.1s finished


In [18]:
print(sklearn_train_pred[:10])

[array([[0., 1., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       ...,
       [0., 1., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 1., 0., 0., 0.]]), array([[1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
     

In [20]:
type_classes= sklearn_pipeline.predict(X[:10])
print(type_classes[:10])

[['house' 'РОССИЯ' 'МОСКВА' 'МОСКВА']
 ['house' 'РОССИЯ' 'МОСКВА' 'МОСКВА']
 ['house' 'РОССИЯ' 'МОСКВА' 'МОСКВА']
 ['subregion' 'РОССИЯ' 'МОСКОВСКАЯ' '0']
 ['city' 'РОССИЯ' 'МОСКОВСКАЯ' 'ХОТЬКОВО']
 ['house' 'РОССИЯ' 'МОСКВА' 'МОСКВА']
 ['house' 'РОССИЯ' 'МОСКВА' 'ТРОИЦК']
 ['house' 'РОССИЯ' 'МОСКВА' 'МОСКВА']
 ['house' 'РОССИЯ' 'МОСКВА' 'МОСКВА']
 ['house' 'РОССИЯ' 'МОСКОВСКАЯ' 'ШАТУРА']]


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.0s finished


In [21]:
import pickle

In [22]:
# save the model to disk
filename = 'model_dj02.sav'
pickle.dump(sklearn_pipeline, open(filename, 'wb'))

# some time later...

# load the model from disk
#loaded_model = pickle.load(open(filename, 'rb'))
#result = loaded_model.score(X_test, Y_test)
#print(result)

In [0]:
!ls

gdrive	model_dj01.sav	sample_data


In [0]:
from google.colab import files
files.download('model_dj01.sav')